In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from nep_lms.embeddings.variants import MiniLML6V3Variant

variant = MiniLML6V3Variant()
variant.train(epochs=10, early_stop=True, early_stop_patience=4, push_to_hub=False)

Filter:   0%|          | 0/669 [00:00<?, ? examples/s]

Filter:   0%|          | 0/34 [00:00<?, ? examples/s]

Filter:   0%|          | 0/134 [00:00<?, ? examples/s]

Computing widget examples:   0%|          | 0/3 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Title Excerpt Loss,With Summary Loss,Ne En Loss,Excerpt Paraphrase Loss
100,0.369200,No log,0.036394,0.013813,0.243402,0.000124
200,0.332900,No log,0.033621,0.012324,0.227694,0.000093
300,0.353400,No log,0.031169,0.010828,0.209932,0.000078
400,0.268600,No log,0.029400,0.009834,0.199097,0.000068
500,0.286700,No log,0.028403,0.009186,0.186802,0.000071
600,0.252000,No log,0.027403,0.008988,0.181903,0.000069
700,0.264000,No log,0.026436,0.008985,0.169967,0.000073
800,0.276400,No log,0.026820,0.008927,0.162206,0.000082


early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_excerpt_paraphrase_loss so early stopping is disabled
early stopping r

In [4]:
from sentence_transformers import SentenceTransformer
nep_mini_lm_v2 = SentenceTransformer("jangedoo/all-MiniLM-L6-v2-nepali")
nep_mini_lm_v3 = SentenceTransformer("jangedoo/all-MiniLM-L6-v3-nepali")
e5_small = SentenceTransformer("intfloat/multilingual-e5-small")

In [10]:
exp = variant.experiment
all_models = {
    'nep_mini_lm_v2': nep_mini_lm_v2,
    'e5_small': e5_small,
    'nep_mini_lm_v3': nep_mini_lm_v3,
    'nep_mini_lm_v3.1': variant.model,
}
exp.compare(models=all_models)
exp.plot_model_comparison()